In [ ]:
from diffusion_policy.dataset.real_image_dataset import RealImageDataset

In [ ]:
dataset_path = "/home/tlips/Code/diffusion_policy/data/demo_place-cb-val"
import os 
os.path.exists(dataset_path)


In [ ]:
n_latency_steps = 0
n_obs_steps = 2
n_action_steps = 8
horizon = 16

shape_meta = {
    "obs": {
        "camera_0": {
            "shape": [3, 224, 224],
            "type": "rgb"
        },
        "camera_1": {
            "shape": [3, 224, 224],
            "type": "rgb"
        },
        "robot_eef_pose_6d_rot": {
            "shape": [9],
            "type": "low_dim"
        },
        "gripper_width" :{
            "shape": [1],
            "type": "low_dim",
        }
    },
    "action": {
        "shape": [10]
    }
}

from omegaconf import OmegaConf
shape_meta = OmegaConf.create(shape_meta)

dataset = RealImageDataset(shape_meta=shape_meta, dataset_path=dataset_path, horizon=horizon, pad_before=n_obs_steps-1+n_latency_steps, pad_after=n_action_steps-1, n_obs_steps=n_obs_steps, n_latency_steps=n_latency_steps, use_cache=True, seed=42, val_ratio=0.00, max_train_episodes=None, delta_action=False)

In [ ]:

dataset.replay_buffer

In [ ]:
dataset.get_validation_dataset().replay_buffer

In [ ]:
print(list(dataset.replay_buffer.meta["episode_ends"]))

In [ ]:
buffer = dataset.replay_buffer
for episode_idx in range(0,len(buffer.meta.episode_ends),10):
    print(f"Episode {episode_idx}")
    episode = buffer.get_episode(episode_idx)
    # get all images "camera_0 from the episode
    images = episode["camera_0"]
    # show first image and last image
    import matplotlib.pyplot as plt
    # side by side
    fig, ax = plt.subplots(1, 3)
    ax[0].imshow(images[0])
    ax[1].imshow(images[-1])
    # display state and action 
    actions = episode["action"]
    eef_pose = episode["robot_eef_pose_6d_rot"]
    print(eef_pose[0])
    ax[2].plot(actions)
    ax[2].plot(eef_pose)
    plt.show()



In [ ]:
# detailed plot of observations vs actions
import numpy as np
import matplotlib.pyplot as plt
buffer = dataset.replay_buffer
episode_idx = 0
episode = buffer.get_episode(episode_idx)
actions = episode["action"]
eef_pose = episode["robot_eef_pose_6d_rot"]
print(eef_pose.shape)
print(actions.shape)
g = episode["gripper_width"]
# add new dim 
print(g.shape)
plt.plot(actions[:, 3])
plt.plot(eef_pose[:, 3])
plt.legend(["action", "eef_pose"])
